In [37]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import pandas as pd
import time
import ast
from pymongo import MongoClient
import urllib.parse
import time
from pymongo import MongoClient
import urllib.parse

In [ ]:
client = MongoClient("mongodb://127.0.0.1:27017")

In [83]:
switch = True
def scrape():
    dataFrame = {'Hash' : [], 'Time' : [], 'BTC-value' : [], 'USD-value' : []}
    frame = pd.DataFrame(data=dataFrame)
    

    #get the data
    link = "https://www.blockchain.com/btc/unconfirmed-transactions"
    page = requests.get(link)
    soup = bs(page.content)

    numbers = soup.find_all("span", {'class' : 'sc-1ryi78w-0 cILyoi sc-16b9dsl-1 ZwupP u3ufsr-0 eQTRKC'})
    hash = soup.find_all("a", {'class' : 'sc-1r996ns-0 fLwyDF sc-1tbyx6t-1 kCGMTY iklhnl-0 eEewhk d53qjk-0 ctEFcK'})


    #Make the arrays for the values we need
    hash_value = [content.get_text() for content in hash]
    time_value = [content.get_text() for content in numbers[0::3]]
    BTC_value = [content.get_text() for content in numbers[1::3]]
    USD_value = [content.get_text() for content in numbers[2::3]]

    #Make dataframe
    dataFrame = {'Hash' : hash_value, 'Time' : time_value, 'BTC-value' : BTC_value, 'USD-value' : USD_value}
    frame = pd.DataFrame(dataFrame)
    frame_complete = frame

    scrape.appendedDF = frame.append(frame_complete).drop_duplicates()
    frame = scrape.appendedDF
    return frame
    
while True:
    x = scrape()

KeyboardInterrupt: 

In [87]:
df = x

In [84]:
#data cleaning
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].map(lambda x: x.rstrip(' BTC'))
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].str.replace(r'\,', '')

scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].map(lambda x: x.lstrip('$'))
scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].str.replace(r'\,', '')

scrape.appendedDF['USD-value'] = scrape.appendedDF['USD-value'].astype('float')
scrape.appendedDF['BTC-value'] = scrape.appendedDF['BTC-value'].astype('float')

In [86]:
y = df.to_dict('records')
#dingen
hashes_db = client["Hashes"]
col_hashes = hashes_db["Hashes"]
x = col_hashes.insert_one(y[0])